In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import seaborn as sn
import pandas as pd
import altair as alt
import numpy as np
import statsmodels.api as sm
import scipy.stats as sct
import matplotlib.pyplot as plt
import matplotlib
import pandas_profiling
import scipy
import pylab
import io
import math
# Comment this if the data visualisations doesn't work on your side
%matplotlib inline
plt.style.use('bmh')

# Client List

#### Get clients from  market and add  label:
* 1 for Clients from Portfolio 1
* 2 for Clients from Portfolio 2
* 3 for Clients from Portfolio 3
* 0 for Clients from none above. 

In [3]:
market = pd.read_csv("estaticos_market.csv")
market['portfolio'] = 0
market.head()

,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio
0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,...,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0,0
1,1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


### DROP NA FEATURES ABOVE THRESHOLD

* Drop features with missing values greater than 85% of total.

In [4]:
row, col = market.shape
print("%d rows" %(row))
print("%d cols" %(col))

market_na = pd.DataFrame(market.isnull().sum(), columns = ["NA"])
print("Features with missing values greater than 85% of total:")
na_feat_list = (market_na[market_na['NA'] > ((row * 85)/100)])
na_feat_list = na_feat_list.index.values
market = market.drop(na_feat_list,axis = 1)
print(na_feat_list)


462298 rows
183 cols
Features with missing values greater than 85% of total:
['vl_total_tancagem' 'vl_total_veiculos_antt' 'vl_total_veiculos_leves'
 'vl_total_veiculos_pesados' 'qt_art' 'vl_total_tancagem_grupo'
 'vl_total_veiculos_antt_grupo' 'vl_potenc_cons_oleo_gas'
 'de_indicador_telefone' 'vl_frota' 'qt_socios_st_suspensa'
 'qt_socios_pep' 'qt_alteracao_socio_total' 'qt_alteracao_socio_90d'
 'qt_alteracao_socio_180d' 'qt_alteracao_socio_365d' 'qt_socios_pj_ativos'
 'qt_socios_pj_nulos' 'qt_socios_pj_baixados' 'qt_socios_pj_suspensos'
 'qt_socios_pj_inaptos' 'vl_idade_media_socios_pj'
 'vl_idade_maxima_socios_pj' 'vl_idade_minima_socios_pj' 'qt_coligados'
 'qt_socios_coligados' 'qt_coligados_matriz' 'qt_coligados_ativo'
 'qt_coligados_baixada' 'qt_coligados_inapta' 'qt_coligados_suspensa'
 'qt_coligados_nula' 'idade_media_coligadas' 'idade_maxima_coligadas'
 'idade_minima_coligadas' 'coligada_mais_nova_ativa'
 'coligada_mais_antiga_ativa' 'idade_media_coligadas_ativas'
 'coligada_

###  Clients on Portfolio1

In [33]:
data1 = pd.read_csv("estaticos_portfolio1.csv")
portfolio1 = market.loc[market['id'].isin(data1['id'])]
portfolio1['portfolio']=1
portfolio1.shape

/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(555, 76)

###  Clients on Portfolio 2

In [27]:
data2 = pd.read_csv("estaticos_portfolio2.csv")
portfolio2 = market.loc[market['id'].isin(data2['id'])]
portfolio2['portfolio']=2
portfolio2.shape

/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(566, 76)

### Clients on Portfolio 3

In [29]:
data3 = pd.read_csv("estaticos_portfolio3.csv")
portfolio3 = market.loc[market['id'].isin(data3['id'])]
portfolio3['portfolio']=3
portfolio3.shape

/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(265, 76)

### Clients on all Portfolios

In [30]:
portfolio_list = pd.concat([portfolio1,portfolio2,portfolio3], ignore_index=True)
portfolio_list['portfolio'] = 1
portfolio_list = portfolio_list[portfolio1.columns]
portfolio_list= portfolio_list.drop_duplicates(subset=['id'], keep='first')
portfolio_list.shape

(1278, 76)

## Clients on market only

In [31]:
client_list = market[~market['id'].isin(portfolio_list['id'])]
client_list = client_list[portfolio1.columns]
client_list.shape

(461020, 76)

## Clients on market excpet portfolio1

In [25]:
client_list1 = market[~market['id'].isin(portfolio1['id'])]
client_list1 = client_list1[market.columns]
client_list1.shape

(461743, 76)

## Clients on market excpet portfolio2

In [24]:
client_list2 = market[~market['id'].isin(portfolio2['id'])]
client_list2 = client_list2[market.columns]
client_list2.shape

(461732, 76)

## Clients on market excpet portfolio3

In [26]:
client_list3 = market[~market['id'].isin(portfolio3['id'])]
client_list3 = client_list3[market.columns]
client_list3.shape

(462033, 76)

# Portfolio1

1. Fill NA of objects and bool with Mode.

In [46]:
def set_fillna(df):
    str_cols = df.columns[df.dtypes==object]
    for col in str_cols:
        df[col].fillna(df[col].mode().tolist()[0],inplace=True)

set_fillna(client_list1)
set_fillna(client_list2)
set_fillna(client_list3)
set_fillna(portfolio1)
client_list1[client_list1.select_dtypes(include= ['object']).columns].isna().any()

id                                     False
de_natureza_juridica                   False
sg_uf                                  False
natureza_juridica_macro                False
de_ramo                                False
setor                                  False
idade_emp_cat                          False
dt_situacao                            False
fl_rm                                  False
nm_divisao                             False
nm_segmento                            False
sg_uf_matriz                           False
de_saude_tributaria                    False
de_saude_rescencia                     False
de_nivel_atividade                     False
nm_meso_regiao                         False
nm_micro_regiao                        False
de_faixa_faturamento_estimado          False
de_faixa_faturamento_estimado_grupo    False
dtype: bool

In [21]:
portfolio1.select_dtypes(include= ['object']).isna().any()

portfolio1.select_dtypes(include= ['object']).isna().any()
portfolio1.loc[:,'fl_optante_simples'].fillna(portfolio1['fl_optante_simples'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'fl_optante_simei'].fillna(portfolio1['fl_optante_simei'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'de_saude_tributaria'].fillna(portfolio1['de_saude_tributaria'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'de_saude_rescencia'].fillna(portfolio1['de_saude_rescencia'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'de_nivel_atividade'].fillna(portfolio1['de_nivel_atividade'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'nm_meso_regiao'].fillna(portfolio1['nm_meso_regiao'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'nm_micro_regiao'].fillna(portfolio1['nm_micro_regiao'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'de_faixa_faturamento_estimado'].fillna(portfolio1['de_faixa_faturamento_estimado'].mode().tolist()[0],inplace=True)
portfolio1.loc[:,'de_faixa_faturamento_estimado_grupo'].fillna(portfolio1['de_faixa_faturamento_estimado_grupo'].mode().tolist()[0],inplace=True)
portfolio1[portfolio1.select_dtypes(include= ['object']).columns].isna().any()



/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/venv/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


id                                     False
de_natureza_juridica                   False
sg_uf                                  False
natureza_juridica_macro                False
de_ramo                                False
setor                                  False
idade_emp_cat                          False
dt_situacao                            False
fl_rm                                  False
nm_divisao                             False
nm_segmento                            False
fl_spa                                 False
fl_antt                                False
fl_veiculo                             False
sg_uf_matriz                           False
de_saude_tributaria                    False
de_saude_rescencia                     False
de_nivel_atividade                     False
fl_simples_irregular                   False
nm_meso_regiao                         False
nm_micro_regiao                        False
fl_passivel_iss                        False
de_faixa_f

# Portfolio2

1. Fill NA of objects and bool with Mode.

In [22]:
portfolio2.select_dtypes(include= ['object']).isna().any()
portfolio2.loc[:,'nm_meso_regiao'].fillna(portfolio2['nm_meso_regiao'].mode().tolist()[0],inplace=True)
portfolio2.loc[:,'nm_micro_regiao'].fillna(portfolio2['nm_micro_regiao'].mode().tolist()[0],inplace=True)
portfolio2[portfolio2.select_dtypes(include= ['object']).columns].isna().any()

id                                     False
de_natureza_juridica                   False
sg_uf                                  False
natureza_juridica_macro                False
de_ramo                                False
setor                                  False
idade_emp_cat                          False
dt_situacao                            False
fl_rm                                  False
nm_divisao                             False
nm_segmento                            False
fl_spa                                 False
fl_antt                                False
fl_veiculo                             False
fl_optante_simples                     False
fl_optante_simei                       False
sg_uf_matriz                           False
de_saude_tributaria                    False
de_saude_rescencia                     False
de_nivel_atividade                     False
fl_simples_irregular                   False
nm_meso_regiao                         False
nm_micro_r

# Portfolio3

1. Fill NA of objects and bool with Mode.

In [23]:
portfolio3[portfolio3.select_dtypes(include= ['object']).columns].isna().any()

portfolio3.loc[:,'fl_optante_simples'].fillna(portfolio3['fl_optante_simples'].mode().tolist()[0],inplace=True)
portfolio3.loc[:,'fl_optante_simei'].fillna(portfolio3['fl_optante_simei'].mode().tolist()[0],inplace=True)

portfolio3.loc[:,'nm_meso_regiao'].fillna(portfolio3['nm_meso_regiao'].mode().tolist()[0],inplace=True)
portfolio3.loc[:,'nm_micro_regiao'].fillna(portfolio3['nm_micro_regiao'].mode().tolist()[0],inplace=True)

portfolio3.loc[:,'nm_meso_regiao'].fillna(portfolio3['nm_meso_regiao'].mode().tolist()[0],inplace=True)
portfolio3.loc[:,'nm_micro_regiao'].fillna(portfolio3['nm_micro_regiao'].mode().tolist()[0],inplace=True)
portfolio3[portfolio2.select_dtypes(include= ['object']).columns].isna().any()

id                                     False
de_natureza_juridica                   False
sg_uf                                  False
natureza_juridica_macro                False
de_ramo                                False
setor                                  False
idade_emp_cat                          False
dt_situacao                            False
fl_rm                                  False
nm_divisao                             False
nm_segmento                            False
fl_spa                                 False
fl_antt                                False
fl_veiculo                             False
fl_optante_simples                     False
fl_optante_simei                       False
sg_uf_matriz                           False
de_saude_tributaria                    False
de_saude_rescencia                     False
de_nivel_atividade                     False
fl_simples_irregular                   False
nm_meso_regiao                         False
nm_micro_r

# Portfolio1 - Pipeline

1. Fill NA with median for numeric values
2. Standardize numeric cols
3. Normalize numeric cols

In [24]:
portfolio1_num = portfolio1[portfolio1.select_dtypes(include= ['int64','float64']).columns]
portfolio1_num.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio
369,369,0.646575,0.0,0.0,NaN,379.96,1.0,1.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
604,604,6.586301,0.0,0.0,22.0,1865.61,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,2.0,NaN,-100.0,0.0,4,1
1703,1703,8.010959,0.0,0.0,24.0,798.12,1.0,1.0,0.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2521,2521,20.863014,0.0,0.0,26.0,608.47,1.0,1.0,0.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
5788,5788,18.726027,0.0,0.0,23.0,880.71,2.0,2.0,0.0,33.0,...,91.2,19.333333,14.0,15.0,25.0,-6.666667,-44.0,0.0,3,1


In [26]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(portfolio1_num.drop('portfolio', axis = 1))
portfolio1_transformed =  pd.DataFrame(pipeline_transformation, columns=portfolio1_num.drop('portfolio', axis = 1).columns)
portfolio1_transformed['portfolio'] = 1
portfolio1_transformed['id'] = portfolio1['id'].tolist()
portfolio1_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.012525,0.0,0.0,0.340426,0.044267,0.000,0.000,0.0,0.510204,...,0.10483,0.014286,0.013333,0.027027,0.333333,0.111111,0.0,0.000000,1,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...
1,0.000509,0.137662,0.0,0.0,0.319149,0.288152,0.000,0.000,0.0,0.442177,...,0.10483,0.000000,0.000000,0.027027,0.333333,0.000000,0.0,0.010695,1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...
2,0.002889,0.167677,0.0,0.0,0.361702,0.112913,0.000,0.000,0.0,0.632653,...,0.10483,0.014286,0.013333,0.027027,0.333333,0.111111,0.0,0.000000,1,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...
3,0.004660,0.438442,0.0,0.0,0.404255,0.081780,0.000,0.000,0.0,0.442177,...,0.10483,0.014286,0.013333,0.027027,0.333333,0.111111,0.0,0.000000,1,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...
4,0.011734,0.393420,0.0,0.0,0.340426,0.126471,0.125,0.125,0.0,0.346939,...,0.09013,0.200000,0.200000,0.337838,0.311111,0.062222,0.0,0.008021,1,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...


# Portfolio2 - Pipeline

1. Fill NA with median for numeric values
2. Standardize numeric cols
3. Normalize numeric cols

In [27]:
portfolio2_num = portfolio2[portfolio2.select_dtypes(include= ['int64','float64']).columns]
portfolio2_num.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio
1646,1646,11.243836,0.0,4.0,23.0,2791.15,NaN,NaN,NaN,NaN,...,357.066667,25.533333,1361.0,1361.0,1383.0,0.000000,-1.590745,0.000000,0,2
2696,2696,43.789041,16.0,20.0,23.0,1007.87,NaN,NaN,NaN,NaN,...,526.133333,23.200000,991.0,991.0,1306.0,0.000000,-24.119449,0.000000,0,2
3681,3681,16.389041,1.0,1.0,23.0,688.31,2.0,2.0,0.0,70.000000,...,149.033333,4.866667,119.0,104.0,113.0,14.423077,5.309735,6.250000,0,2
4564,4564,39.671233,0.0,0.0,23.0,1397.47,NaN,NaN,NaN,NaN,...,555.766667,28.300000,1829.0,1829.0,1875.0,0.000000,-2.453333,0.000000,1,2
6066,6066,20.268493,3.0,12.0,23.0,707.20,4.0,4.0,0.0,48.666667,...,134.833333,2.933333,177.0,130.0,120.0,36.153846,47.500000,41.042345,0,2


In [28]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(portfolio2_num.drop('portfolio', axis = 1))
portfolio2_transformed =  pd.DataFrame(pipeline_transformation, columns=portfolio2_num.drop('portfolio', axis = 1).columns)
portfolio2_transformed['portfolio'] = 2
portfolio2_transformed['id'] = portfolio2['id'].tolist()
portfolio2_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.026736,0.000000,0.010782,0.071429,0.308166,0.007246,0.007246,0.0,0.450000,...,0.057957,0.074989,0.075851,0.075384,0.174344,0.004790,0.000000,0.000000,2,09e95c1a84049001d086470a8f320a19b076f955a89122...
1,0.002282,0.748557,0.024133,0.053908,0.071429,0.090804,0.007246,0.007246,0.0,0.450000,...,0.052227,0.054360,0.055230,0.071187,0.174344,0.003531,0.000000,0.000000,2,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...
2,0.004422,0.140852,0.001508,0.002695,0.071429,0.051853,0.007246,0.007246,0.0,0.714286,...,0.007204,0.005743,0.005796,0.006159,0.205989,0.005176,0.046684,0.000000,2,16843c9ffb920170477118933798510d8d5f00f5c56c12...
3,0.006341,0.657228,0.000000,0.000000,0.071429,0.138292,0.007246,0.007246,0.0,0.450000,...,0.064751,0.101082,0.101934,0.102202,0.174344,0.004742,0.000000,0.005814,2,ff045934d90aab184dd15e66daf3b3c97193bda5449298...
4,0.009605,0.226894,0.004525,0.032345,0.071429,0.054155,0.021739,0.021739,0.0,0.409524,...,0.002456,0.008976,0.007245,0.006541,0.253667,0.007535,0.306566,0.000000,2,a0ce528caab2c62497a548d0ebd9e935143ef81ed7dbf0...


# Portfolio3 - Pipeline

1. Fill NA with median for numeric values
2. Standardize numeric cols
3. Normalize numeric cols

In [29]:
portfolio3_num = portfolio3[portfolio3.select_dtypes(include= ['int64','float64']).columns]
portfolio3_num.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio
3116,3116,21.813699,2.0,5.0,26.0,644.27,NaN,NaN,NaN,NaN,...,259.166667,40.500000,745.0,745.0,745.0,0.000000,0.000000,0.00,2,3
3681,3681,16.389041,1.0,1.0,23.0,688.31,2.0,2.0,0.0,70.0,...,149.033333,4.866667,119.0,104.0,113.0,14.423077,5.309735,6.25,0,3
4784,4784,40.380822,0.0,0.0,23.0,1881.35,NaN,NaN,NaN,NaN,...,557.866667,24.200000,170.0,170.0,177.0,0.000000,-3.954802,0.00,0,3
7429,7429,43.854795,2.0,1.0,23.0,353.66,NaN,NaN,NaN,NaN,...,506.966667,31.333333,151.0,151.0,151.0,0.000000,0.000000,0.00,0,3
8001,8001,43.865753,0.0,0.0,23.0,NaN,NaN,NaN,NaN,NaN,...,556.833333,34.033333,229.0,229.0,418.0,0.000000,-45.215311,0.00,1,3


In [30]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(portfolio3_num.drop('portfolio', axis = 1))
portfolio3_transformed =  pd.DataFrame(pipeline_transformation, columns=portfolio3_num.drop('portfolio', axis = 1).columns)
portfolio3_transformed['portfolio'] = 3
portfolio3_transformed['id'] = portfolio3['id'].tolist()
portfolio3_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.412409,0.009302,0.013477,0.425,0.057328,0.007246,0.007246,0.0,0.534884,...,0.349667,0.175798,0.189686,0.175583,0.002328,0.004944,0.000000,0.011628,3,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...
1,0.001237,0.306140,0.004651,0.002695,0.350,0.064607,0.007246,0.007246,0.0,0.767442,...,0.026316,0.013243,0.026040,0.026632,0.003461,0.005360,0.061828,0.000000,3,16843c9ffb920170477118933798510d8d5f00f5c56c12...
2,0.003651,0.776138,0.000000,0.000000,0.350,0.261789,0.007246,0.007246,0.0,0.534884,...,0.201754,0.026487,0.042890,0.041716,0.002328,0.004634,0.000000,0.000000,3,1b8d092daff12bed8b3c2ab93bf5df2921865a68aca185...
3,0.009439,0.844193,0.009302,0.002695,0.350,0.009297,0.007246,0.007246,0.0,0.534884,...,0.266485,0.021553,0.038039,0.035588,0.002328,0.004944,0.000000,0.000000,3,3aafa3a1f12f483740a52956af100223333d0e3c79bf2c...
4,0.010691,0.844407,0.000000,0.000000,0.350,0.128068,0.007246,0.007246,0.0,0.534884,...,0.290986,0.041807,0.057953,0.098515,0.002328,0.001401,0.000000,0.005814,3,c60431fbca90067f105570e1e8a684bcd7048205bbf6c4...


## CLIENT LIST - PIPELINE

In [37]:
client_num = client_list[client_list.select_dtypes(include= ['int64','float64']).columns]
client_num.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio
0,0,14.457534,0.0,0.0,23.0,NaN,2.0,2.0,0.0,44.0,...,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0,0
1,1,1.463014,0.0,0.0,9.0,468.93,1.0,1.0,0.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2,7.093151,0.0,0.0,26.0,713.50,1.0,1.0,0.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,3,6.512329,0.0,0.0,26.0,475.73,1.0,1.0,0.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,3.200000,0.0,0.0,27.0,2874.04,1.0,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [39]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(client_num.drop('portfolio', axis = 1))
client_transformed =  pd.DataFrame(pipeline_transformation, columns=client_num.drop('portfolio', axis = 1).columns)
client_transformed['portfolio'] = 0
client_transformed['id'] = client_list['id'].tolist()
client_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.135704,0.0,0.0,0.348485,0.011285,0.004082,0.008130,0.0,0.356589,...,0.003381,0.000504,0.000504,0.000498,0.003584,0.001102,0.0,0.0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...


## CLIENT LIST - Except Porfolios * PIPELINE

In [49]:
client_num1 = client_list1[client_list.select_dtypes(include= ['int64','float64']).columns]
print(client_num1.shape)

client_num2 = client_list2[client_list.select_dtypes(include= ['int64','float64']).columns]
print(client_num2.shape)

client_num3 = client_list3[client_list.select_dtypes(include= ['int64','float64']).columns]
print(client_num3.shape)

(461743, 41)
(461732, 41)
(462033, 41)


In [52]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(client_num1.drop('portfolio', axis = 1))
client_transformed1 =  pd.DataFrame(pipeline_transformation, columns=client_num1.drop('portfolio', axis = 1).columns)
client_transformed1['portfolio'] = 0
client_transformed1['id'] = client_list1['id'].tolist()
client_transformed1.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.135704,0.0,0.0,0.348485,0.011291,0.004082,0.008130,0.0,0.356589,...,0.003381,0.000504,0.000504,0.000498,0.003584,0.001102,0.0,0.0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...


In [53]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(client_num2.drop('portfolio', axis = 1))
client_transformed2 =  pd.DataFrame(pipeline_transformation, columns=client_num2.drop('portfolio', axis = 1).columns)
client_transformed2['portfolio'] = 0
client_transformed2['id'] = client_list2['id'].tolist()
client_transformed2.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.135704,0.0,0.0,0.348485,0.011285,0.004082,0.008130,0.0,0.356589,...,0.003381,0.000504,0.000504,0.000498,0.003584,0.001102,0.0,0.0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...


In [54]:
num_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),("scaler", StandardScaler(copy=True, with_mean=True, with_std=True)),("minmaxscaler", MinMaxScaler(copy=True, feature_range=(0, 1)))])
pipeline_transformation = num_pipeline.fit_transform(client_num3.drop('portfolio', axis = 1))
client_transformed3 =  pd.DataFrame(pipeline_transformation, columns=client_num3.drop('portfolio', axis = 1).columns)
client_transformed3['portfolio'] = 0
client_transformed3['id'] = client_list3['id'].tolist()
client_transformed3.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,portfolio,id
0,0.000000,0.135704,0.0,0.0,0.348485,0.011290,0.004082,0.008130,0.0,0.356589,...,0.003381,0.000504,0.000504,0.000498,0.003584,0.001102,0.0,0.0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,0.004172,0.000039,0.000039,0.000037,0.003584,0.001144,0.0,0.0,0,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...


## One hot encoding - Client List 1, 2 and 3

In [ ]:
set_fillna(client_list1)
set_fillna(client_list2)
set_fillna(client_list3)
set_fillna(portfolio1)
client_list1[client_list1.select_dtypes(include= ['object']).columns].isna().any()

In [60]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(client_list1['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(client_list1['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(client_list1['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(client_list1['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(client_list1['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [61]:
client_transformed1 = pd.concat([client_transformed3, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
client_transformed1.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_MUITO BAIXA,x0_AMARELO,x0_AZUL,x0_CINZA,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.135704,0.0,0.0,0.348485,0.011290,0.004082,0.008130,0.0,0.356589,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [62]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(client_list2['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(client_list2['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(client_list2['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(client_list2['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(client_list2['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [63]:
client_transformed2 = pd.concat([client_transformed2, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
client_transformed2.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_MUITO BAIXA,x0_AMARELO,x0_AZUL,x0_CINZA,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.135704,0.0,0.0,0.348485,0.011285,0.004082,0.008130,0.0,0.356589,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [64]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(client_list3['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(client_list3['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(client_list3['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(client_list3['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(client_list3['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [65]:
client_transformed3 = pd.concat([client_transformed3, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
client_transformed3.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_MUITO BAIXA,x0_AMARELO,x0_AZUL,x0_CINZA,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.135704,0.0,0.0,0.348485,0.011290,0.004082,0.008130,0.0,0.356589,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## One hot encoding - Client

In [55]:
client_obj = client_list[client_list.select_dtypes(include= ['object']).columns]


In [56]:
client_obj['setor'].fillna(client_obj['setor'].mode().tolist()[0],inplace=True)
client_obj['dt_situacao'].fillna(client_obj['dt_situacao'].mode().tolist()[0],inplace=True)
client_obj['nm_segmento'].fillna(client_obj['nm_segmento'].mode().tolist()[0],inplace=True)
client_obj['fl_spa'].fillna(client_obj['fl_spa'].mode().tolist()[0],inplace=True)
client_obj['nm_divisao'].fillna(client_obj['nm_divisao'].mode().tolist()[0],inplace=True)
client_obj['fl_antt'].fillna(client_obj['fl_antt'].mode().tolist()[0],inplace=True)
client_obj['fl_veiculo'].fillna(client_obj['fl_veiculo'].mode().tolist()[0],inplace=True)
client_obj['fl_optante_simples'].fillna(client_obj['fl_optante_simples'].mode().tolist()[0],inplace=True)
client_obj['fl_optante_simei'].fillna(client_obj['fl_optante_simei'].mode().tolist()[0],inplace=True)
client_obj['sg_uf_matriz'].fillna(client_obj['sg_uf_matriz'].mode().tolist()[0],inplace=True)
client_obj['de_saude_tributaria'].fillna(client_obj['de_saude_tributaria'].mode().tolist()[0],inplace=True)
client_obj['de_saude_rescencia'].fillna(client_obj['de_saude_rescencia'].mode().tolist()[0],inplace=True)
client_obj['de_nivel_atividade'].fillna(client_obj['de_nivel_atividade'].mode().tolist()[0],inplace=True)
client_obj['fl_simples_irregular'].fillna(client_obj['fl_simples_irregular'].mode().tolist()[0],inplace=True)
client_obj['nm_meso_regiao'].fillna(client_obj['nm_meso_regiao'].mode().tolist()[0],inplace=True)
client_obj['nm_micro_regiao'].fillna(client_obj['nm_micro_regiao'].mode().tolist()[0],inplace=True)
client_obj['fl_passivel_iss'].fillna(client_obj['fl_passivel_iss'].mode().tolist()[0],inplace=True)
client_obj['de_faixa_faturamento_estimado'].fillna(client_obj['de_faixa_faturamento_estimado'].mode().tolist()[0],inplace=True)
client_obj['de_faixa_faturamento_estimado_grupo'].fillna(client_obj['de_faixa_faturamento_estimado_grupo'].mode().tolist()[0],inplace=True)


/home/marcelo/Documents/codenation_squad4/squad-4-ad-data-science-1/venv/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [57]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(client_obj['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(client_obj['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(client_obj['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(client_obj['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(client_obj['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [58]:
client_transformed = pd.concat([client_transformed, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
client_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_MUITO BAIXA,x0_AMARELO,x0_AZUL,x0_CINZA,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.135704,0.0,0.0,0.348485,0.011285,0.004082,0.008130,0.0,0.356589,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000002,0.013594,0.0,0.0,0.136364,0.004919,0.000000,0.004065,0.0,0.224806,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.000004,0.066500,0.0,0.0,0.393939,0.008181,0.000000,0.004065,0.0,0.263566,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.000006,0.061042,0.0,0.0,0.393939,0.005010,0.000000,0.004065,0.0,0.294574,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.000009,0.029916,0.0,0.0,0.409091,0.036990,0.000000,0.004065,0.0,0.341085,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# One hot encoding - Portfolio 1, 2 e 3

In [31]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(portfolio1['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(portfolio1['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(portfolio1['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(portfolio1['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(portfolio1['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [32]:
portfolio1_transformed = pd.concat([portfolio1_transformed, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
portfolio1_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_MUITO BAIXA,x0_AMARELO,x0_AZUL,x0_CINZA,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.012525,0.0,0.0,0.340426,0.044267,0.000,0.000,0.0,0.510204,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.000509,0.137662,0.0,0.0,0.319149,0.288152,0.000,0.000,0.0,0.442177,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.002889,0.167677,0.0,0.0,0.361702,0.112913,0.000,0.000,0.0,0.632653,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.004660,0.438442,0.0,0.0,0.404255,0.081780,0.000,0.000,0.0,0.442177,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.011734,0.393420,0.0,0.0,0.340426,0.126471,0.125,0.125,0.0,0.346939,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(portfolio2['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(portfolio2['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(portfolio2['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(portfolio2['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(portfolio2['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [34]:
portfolio2_transformed = pd.concat([portfolio2_transformed, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
portfolio2_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ENTIDADES SEM FINS LUCRATIVOS,x0_OUTROS,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_AMARELO,x0_AZUL,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.026736,0.000000,0.010782,0.071429,0.308166,0.007246,0.007246,0.0,0.450000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.002282,0.748557,0.024133,0.053908,0.071429,0.090804,0.007246,0.007246,0.0,0.450000,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.004422,0.140852,0.001508,0.002695,0.071429,0.051853,0.007246,0.007246,0.0,0.714286,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.006341,0.657228,0.000000,0.000000,0.071429,0.138292,0.007246,0.007246,0.0,0.450000,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.009605,0.226894,0.004525,0.032345,0.071429,0.054155,0.021739,0.021739,0.0,0.409524,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
enc = OneHotEncoder()
#client_id = enc.fit_transform(data_object['sg_uf'].values.reshape(-1, 1)).toarray()
#print(client_id[1])
de_ramo_one_hot =  enc.fit_transform(portfolio3['de_ramo'].values.reshape(-1, 1)).toarray()
de_ramo_df = pd.DataFrame(de_ramo_one_hot, columns=enc.get_feature_names().tolist())

setor_one_hot = enc.fit_transform(portfolio3['setor'].values.reshape(-1, 1)).toarray()
setor_df = pd.DataFrame(setor_one_hot, columns=enc.get_feature_names().tolist())

natureza_juridica_macro_one_hot = enc.fit_transform(portfolio3['natureza_juridica_macro'].values.reshape(-1, 1)).toarray()
natureza_juridica_macro_df = pd.DataFrame(natureza_juridica_macro_one_hot, columns=enc.get_feature_names().tolist())

de_nivel_atividade_one_hot = enc.fit_transform(portfolio3['de_nivel_atividade'].values.reshape(-1, 1)).toarray()
de_nivel_atividade_df = pd.DataFrame(de_nivel_atividade_one_hot, columns=enc.get_feature_names().tolist())


de_saude_tributaria_one_hot = enc.fit_transform(portfolio3['de_saude_tributaria'].values.reshape(-1, 1)).toarray()
de_saude_tributaria_df = pd.DataFrame(de_saude_tributaria_one_hot, columns=enc.get_feature_names().tolist())


In [36]:
portfolio3_transformed = pd.concat([portfolio3_transformed, de_ramo_df,setor_df,natureza_juridica_macro_df,de_nivel_atividade_df,de_saude_tributaria_df], axis=1, sort=False)
portfolio3_transformed.head()

,Unnamed: 0,idade_empresa_anos,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,nu_meses_rescencia,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,...,x0_ENTIDADES SEM FINS LUCRATIVOS,x0_OUTROS,x0_ALTA,x0_BAIXA,x0_MEDIA,x0_AZUL,x0_CINZA,x0_LARANJA,x0_VERDE,x0_VERMELHO
0,0.000000,0.412409,0.009302,0.013477,0.425,0.057328,0.007246,0.007246,0.0,0.534884,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.001237,0.306140,0.004651,0.002695,0.350,0.064607,0.007246,0.007246,0.0,0.767442,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.003651,0.776138,0.000000,0.000000,0.350,0.261789,0.007246,0.007246,0.0,0.534884,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.009439,0.844193,0.009302,0.002695,0.350,0.009297,0.007246,0.007246,0.0,0.534884,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.010691,0.844407,0.000000,0.000000,0.350,0.128068,0.007246,0.007246,0.0,0.534884,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## LABEL ENCODER

In [59]:
lb_make = LabelEncoder()

portfolio1_transformed["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(portfolio1["de_faixa_faturamento_estimado"])

portfolio2_transformed["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(portfolio2["de_faixa_faturamento_estimado"])

portfolio3_transformed["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(portfolio3["de_faixa_faturamento_estimado"])

client_transformed["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(client_obj["de_faixa_faturamento_estimado"])

client_transformed1["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(client_list1["de_faixa_faturamento_estimado"])

client_transformed2["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(client_list2["de_faixa_faturamento_estimado"])

client_transformed3["de_faixa_faturamento_estimado_encode"] = lb_make.fit_transform(client_list3["de_faixa_faturamento_estimado"])


# PCA

In [39]:
#pca = PCA(n_components=5)
#principalComponents = pca.fit_transform(portfolio1_transformed)
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5'])

In [1]:
#principalDf.head()